In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter

In [3]:
# 1. PDF 파일 로드
# 
loader = PyMuPDFLoader('./도서목록.pdf')  # Document loaders인 PyMuPDFLoader를 사용해서 PDF 파일을 읽을 준비를 한다
documents = loader.load()         	      # load 메서드를 호출하여 PDF 파일을 문서 형식으로 가져온다.

# load() 메소드는 문장의 배열을 리턴한다.
# 하나의 문장을 Document라는 클래스로 표현한다.
# PyMuPDFLoader에서는 1페이지 마다 1개의 Document가 생성된다.
# PyMuPDFLoader에서는 하나의 PDF 파일을 로드하면 몇 페이지 분량의 Document가 배열로 리턴된다.

In [4]:
# 2. 텍스트 분할기 초기화
#
# 텍스트가 너무 길면 모델이 처리할 수 있는 글자수의 한계를 넘계되어서 텍스트를 분할해야한다.
# 텍스트 분할기를 만든다. 텍스트를 분할하기 위한 준비.
text_splitter = SpacyTextSplitter(		
    chunk_size=300, 					# 분할할 때 각 청크의 최대 길이를 300자로 설정
    pipeline='ko_core_news_sm'			# 분할에 사용할 한국어 모델인 ko_core_news_sm
)

# spaCy는 자연어 처리 라이브러리.
# 자연어 처리는 인간에서 사용하는 언어를 컴퓨터가 이해하고 분석할 수 있도록 하는 기술로
# spaCy 에는 문장 분할, 품사 판단, 명사구 추출, 구문 분석 등을 제공한다.

In [5]:
# 3. 문서 분할
# 
# text_splitter를 사용하여 로드된 문서를 여러 조각으로 나눈다.
# 입력으로 받은 documents를 내부적으로 설정된 방식(여기서는 스페이시 한국어 모델과 최대 300자의 청크 크기)을 사용하여 분할합니다.
splitted_documents = text_splitter.split_documents(documents)

Created a chunk of size 531, which is longer than the specified 300


In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [7]:
# 4. 임베딩 생성기 초기화
# OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트 임베딩을 생성하고, 이것을 벡터 스토어에 저장하기 위한 설정
embeddings = OpenAIEmbeddings(
    openai_api_key=key,					# OpenAI API 키를 사용하여 인증
    model='text-embedding-ada-002' 		# OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트 임베딩을 생성
)

C:\Users\User\AppData\Local\Temp\ipykernel_80996\3485767079.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(


In [8]:
# 5. 벡터 스토어 초기화. 
#
# Chroma 벡터 스토어를 초기화하고, 임베딩 데이터를 ./data 디렉토리에 저장합니다.
# 벡터 스토어는 텍스트 임베딩을 저장하고 검색하는 데 사용됩니다.
database = Chroma(							# Chroma 벡터 스토어를 초기화
    persist_directory = './data', 			# 임베딩 데이터를 저장할 디렉토리를 ./data로 설정
    embedding_function = embeddings			# embedding 함수로 위에서 만든 embeddings 객체 지정
)

# 이 과정은 긴 텍스트를 처리하고 검색 가능하도록 임베딩으로 변환하여 저장하는 데 유용합니다. 
# 벡터 스토어를 통해 효율적인 검색과 데이터 관리가 가능해집니다.

C:\Users\User\AppData\Local\Temp\ipykernel_80996\925303431.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  database = Chroma(							# Chroma 벡터 스토어를 초기화


In [9]:
#  6. 분할된 문서를 벡터 스토어에 저장
# 
# database는 Chroma 벡터 스토어 객체이다.
# add_documents() 메서드는 분할된 텍스트 조각을 벡터 스토어에 추가하는 역할
# splitted_documents는 앞서 분할된 텍스트 조각들의 리스트입니다. 이 리스트의 각 항목은 텍스트 청크

# 각 텍스트 청크를 벡터로 변환하고, 변환된 벡터를 벡터 스토어에 저장

database.add_documents(			# 분할된 텍스트 조각들을 데이터베이스에 추가한다
    splitted_documents			# 추가할 문서를 지정
)

print('완료')

완료
